In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from collections import Counter

In [34]:
def open_data(path):
    
    with open(path, encoding='utf-8') as rdata:
        lines = rdata.read().split('\n')
    
    return lines

def process_the_data(path):
    ID = []
    pos = []
    feats = []  # not included
    head = []
    deprel = []
    deprels = []  # target

    lines = open_data(path)
    
    for line in lines:
    
        string = line.split('\t')
    
        if len(string) != 1:
            ID.append(int(string[0]))
            pos.append(string[3])
            feats.append(string[5])
            head.append(int(string[6]))
            deprel.append(string[7])
            deprels.append([''.join(elem.split(':')[1:]) for elem in string[8].split('|')])
            #print([''.join(elem.split(':')[1:]) for elem in string[8].split('|')])
    
    predictors = [pos[:], deprel[:]]
    
    return predictors, deprels[:], ID[:], head[:]

train_x, train_y, train_id, train_head = process_the_data('fipb-ud-train.conllu')
dev_x, dev_y, dev_id, dev_head = process_the_data('fipb-ud-dev.conllu')

In [35]:
names = ["ID", "TOKEN", "LEM", "POS", "POSABBR", "FEATS", "HEAD", "DEPREL", "DEPRELS", "MISC"]

def transform_data(fit, fit_y, predict, predict_y):  # partially void method! beware!
    
    for i in range(len(fit)):
        
        lb = LabelEncoder()
        fit[i] = lb.fit_transform(fit[i])
        predict[i] = lb.transform(predict[i])
        #print(predictors[i])
    
    mlb = MultiLabelBinarizer()
    fit_y = mlb.fit_transform(fit_y)
    predict_y = mlb.transform(predict_y)
    
    return fit_y, predict_y

train_y, dev_y = transform_data(train_x, train_y, dev_x, dev_y)
train_y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [36]:
def make_X(*args):
    X = []
    
    for i in range(len(args[0])):  # assuming they are all even in terms of length
        X.append([])
        for arg in args:
            X[-1].append(arg[i])
    
    return X
            
X = make_X(train_id, train_x[0], train_head, train_x[1])
X_check = make_X(dev_id, dev_x[0], dev_head, dev_x[1])

In [37]:
clf = RandomForestClassifier()
clf.fit(X, train_y)
#train_y

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
print(clf.feature_importances_)

[ 0.10832395  0.23324435  0.1091725   0.54925921]


In [43]:
clf.score(X_check, dev_y)
#print(train_x[0])

0.68737037441327364